In [1]:
import logging
import pickle, random
import torch, os, pytorch_lightning as pl, glob
import torch.distributed as dist
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer
import torch.nn.functional as F
from datasets import load_dataset



In [2]:
# dm = MedMCQA_Datamodule_to_save_datasets()
# dm.setup()

In [3]:
dataset_together = load_dataset("openlifescienceai/medmcqa")

In [4]:
dataset = dataset_together["validation"]

In [5]:
from transformers import AutoTokenizer, AutoModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# del generation_model

In [7]:
# generate prompts from dataset

A,B,C,D = dataset['opa'], dataset['opb'],dataset['opc'],dataset['opd']
questions = dataset['question']
subject_names = dataset['subject_name']
topic_names = dataset['topic_name']



In [8]:
input_prompts_long = list(map(lambda a,b,c,d,q,subject,topic: f"Subject: {subject}, Topic: {topic}\nQuestion: {q}\nA: {a}\nB: {b}\nC: {c}\nD: {d}\nKey concepts: ",
                    A,B,C,D,questions, subject_names, topic_names))

input_prompts = list(map(lambda a,b,c,d,q,subject,topic: f"Subject: {subject}, Topic: {topic}\nQuestion: {q}\n To answer this question, we need to notice that ",
                    A,B,C,D,questions, subject_names, topic_names))

In [9]:
input_prompts_short = list(map(lambda q: f"{q} It is relavent to",
                    questions))



input_prompts_concept = list(map(lambda q: f"{q} Key Concept:",
                    questions))

In [10]:

import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel

import torch
# prompt = ["NMR is used to", 
#           "Which of the following is not true about glomerular capillaries: It is relavent to",
#           "A 29 yrs old woman with a pregnancy of 17 week has a 10 years old boy with down syndrome. She does not want another down syndrome kid; best advice to her is: It is relavent to"]
# Load the pretrained model and tokenizer
model_name = "healx/gpt-2-pubmed-medium"
max_length = 100
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.padding_side='left'
tokenizer.pad_token = tokenizer.eos_token


In [11]:
import tqdm

In [12]:
def generate_datasets_with_prompt_and_save(dataset, save_path):
    
    questions = dataset['question']
    input_prompts_hint = list(map(lambda q: f"{q} Hint:",
                    questions))
    
    for i in tqdm.tqdm(range(len(input_prompts_hint)//16)):
        inputs = tokenizer(input_prompts_hint[16*i:16*(i+1)], return_tensors="pt",  padding=True, truncation=True).to(device)

        # Generate text by feeding the encoded input through the model and sampling output tokens
        outputs = model.generate(input_ids=inputs["input_ids"],
                                attention_mask=inputs["attention_mask"], 
                                max_length=max_length+inputs["input_ids"].shape[1], 
                                num_beams=10, early_stopping=True,
                                repetition_penalty=4.2,
                                pad_token_id=50256
                                )
        generated_text = tokenizer.batch_decode(outputs, 
                                        skip_special_tokens=True
                                        )
        generated_text = generated_text.split('.', 1)[0] + '.'
        break


    print(generated_text)


# # Decode the generated tokens back into textf
# for i in range(len(outputs)):
#     
    


#     print(f"Generated Text:\n{generated_text}\n")
    

In [13]:
inputs.keys(), inputs["input_ids"].shape, inputs["attention_mask"].shape, output

NameError: name 'inputs' is not defined

In [ ]:
print(tokenizer.pad_token_id)

None


In [ ]:
len(input_prompts)

4183

In [ ]:
generation_model.tokenizer.padding_side='left'
if generation_model.tokenizer.pad_token is None:
    generation_model.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    generation_model.model.resize_token_embeddings(len(generation_model.tokenizer))
else:
    print(generation_model.tokenizer.pad_token)
    
generation_model.tokenizer.pad_token = generation_model.tokenizer.eos_token
inputs  = generation_model.tokenizer(input_prompts[0:16], padding=True, truncation=True, return_tensors="pt")


inputs = {k: v.to(device) for k, v in inputs.items()}
output_sequences = generation_model.model.generate(
            input_ids=inputs["input_ids"],
            # attention_mask=inputs["attention_mask"],
            max_length=64 + inputs["input_ids"].shape[1],
            pad_token_id=generation_model.model.config.eos_token_id,  # Ensure pad_token_id is set for open-ended generation
            temperature=1.0,
            top_k=50,
            top_p=0.95,
            repetition_penalty=5.2,
            do_sample=True,
            # num_beams  = 4,
            num_return_sequences=4,
        )


[PAD]


In [ ]:
generation_model.tokenizer.pad_token

'<|endoftext|>'

In [ ]:
output_sequences.shape, inputs["input_ids"].shape

(torch.Size([64, 124]), torch.Size([16, 60]))

In [ ]:
for i, generated_sequence in enumerate(output_sequences):
    decoded_text = generation_model.tokenizer.decode(generated_sequence, skip_special_tokens=True)
    print(f"Generated Text {i+1}: {decoded_text}")
    print()
    if i == 10: break

Generated Text 1: Subject: Physiology, Topic: None
Question: Which of the following is not true for myelinated nerve fibers:
 To answer this question, we need to notice the fact that ery [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]. [PAD]    [PAD] [PAD]  * [PAD] [PAD]  ( [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ). [PAD]   [PAD] [PAD]   [PAD]  ) [PAD]  E [PAD] [PAD] [PAD] [PAD] [PAD]  / [PAD]  ; [PAD]  

Generated Text 2: Subject: Physiology, Topic: None
Question: Which of the following is not true for myelinated nerve fibers:
 To answer this question, we need to notice the fact that  [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]  ( [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [ ]:
print(input_prompts_long[0])

Subject: Physiology, Topic: None
Question: Which of the following is not true for myelinated nerve fibers:
A: Impulse through myelinated fibers is slower than non-myelinated fibers
B: Membrane currents are generated at nodes of Ranvier
C: Saltatory conduction of impulses is seen
D: Local anesthesia is effective only when the nerve is not covered by myelin sheath
Key concepts: 


In [ ]:
torch.cuda.empty_cache()
import gc

gc.collect()


9041